In [1]:
import pandas as pd
from Levenshtein import distance
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import re

In [2]:
df = pd.read_csv('../Data/quadruplet_test_pred.csv')

In [4]:
df.shape

(39, 11)

In [5]:
df['quadruplet'].isnull().sum()

0

In [3]:
def extract_quadruplet(sequence):
    extractions = []
    # find all matching quadruplet with (); pattern
    quadruplets = re.findall("\(.*?\)", sequence)
    for quadruplet in quadruplets:
        # Remove the in the start "("  and at the end ")".
        quadruplet = quadruplet[1:-1]
        try:
            aspect_term, opinion_term, sentiment, aspect_category = quadruplet.split(', ')
        except ValueError:
            aspect_term, opinion_term, sentiment, aspect_category = '', '', '', ''
        aspect_term = aspect_term.strip().lower()
        opinion_term = opinion_term.strip().lower()
        sentiment = sentiment.strip().lower()
        aspect_category = aspect_category.strip().lower()
        extractions.append((aspect_term, opinion_term, sentiment, aspect_category)) 
    return extractions

In [4]:
def correct_sentiment(quadruplet_sent):
    #aspect asli
    aspect_category = ['negative', 'positive', 'neutral']
    min_dist = 9999
    for aspect in aspect_category:
        dist = distance(quadruplet_sent, aspect)
        if dist<min_dist:
            min_dist=dist
            corrected_aspect = aspect
    return corrected_aspect

In [5]:
def correct_aspect(quadruplet_aspect):
    #aspect asli
    aspect_category = ['price', 'produk', 'payment', 'website&apps', 'delivery', 'customerservice']
    min_dist = 9999
    for aspect in aspect_category:
        dist = distance(quadruplet_aspect, aspect)
        if dist<min_dist:
            min_dist=dist
            corrected_aspect = aspect
    return corrected_aspect

In [6]:
def post_process(quad):
    aspect_term, opinion_term, sentiment, aspect_category = quad
    aspect_term = correct_aspect(aspect_term)
    sentiment = correct_sentiment(sentiment)
    return aspect_term, opinion_term, sentiment, aspect_category

In [21]:
sents = []
aspects = []
for i in range(len(df)):
    row = df.iloc[i]
    quads = extract_quadruplet(row['quadruplet'])
    for quad in quads:
        aspect_term, opinion_term, sentiment, aspect_category = post_process(quad)
        if sentiment not in sents:
            sents.append(sentiment)
        if aspect_category not in aspects:
            aspects.append(aspect_category)

In [22]:
print(sents)
print(aspects)

['negative', 'positive', 'neutral']
['website&apps', 'customerservice', 'delivery', 'price', 'payment', 'product']


# Label diff

In [26]:
df.columns

Index(['original_id', 'content', 'clean_tweet', 'final_sentiment', 'labels',
       'quadruplet', 'spam', 'sentiment_label', 'pred_quadruplet_pt_bart',
       'pred_quadruplet_pt_t5', 'pred_quadruplet_tf_t5'],
      dtype='object')

In [9]:
pred_quadruplet_col = 'pred_quadruplet_pt_t5'
ori_quadruplet_col = 'quadruplet'

In [10]:
aspect_categories = ['price', 'produk', 'payment', 'website&apps', 'delivery', 'customerservice']
annot_errors = []
model_errors = []
df['annotated_multilabel'] = ''
df['keyword_multilabel'] = ''
df['model_multilabel'] = ''
for i in range(len(df)):
    row = df.iloc[i]
    #for annotated aspect categories
    annotated_quadruplets = extract_quadruplet(row[ori_quadruplet_col])
    annotated_aspect = []
    #for model aspect categories
    model_quadruplets = extract_quadruplet(row[pred_quadruplet_col])
    model_aspect = []
    #for annotated quadruplet
    for quadruplet in annotated_quadruplets:
        #for annotated
        aspect_term, opinion_term, sentiment, aspect_category = post_process(quadruplet)
        annotated_aspect.append(aspect_category)
        #annotated_aspect.append(aspect_categories.index(aspect_category))
    #for model quadruplet
    for quadruplet in model_quadruplets:
        #for annotated
        aspect_term, opinion_term, sentiment, aspect_category = post_process(quadruplet)
        model_aspect.append(aspect_category)
        #model_aspect.append(aspect_categories.index(aspect_category))
    #for annotated label
    df.at[i, 'annotated_multilabel'] = set(annotated_aspect)
    #for model label
    df.at[i, 'model_multilabel'] = set(model_aspect)
    #for keyword label
    labels = row['labels'].split(';')
    keyword_multilabels = [label.replace(' ', '') for label in labels if label != '']
    #keyword_multilabels = [aspect_categories.index(keyword_multilabels) for label in labels]
    df.at[i, 'keyword_multilabel'] = set(keyword_multilabels)

In [18]:
df.head()[[ori_quadruplet_col, pred_quadruplet_col, 'labels']]

,quadruplet,pred_quadruplet_pt_t5,labels
0,"(pesananku, udah bayar tapi kenapa dibatalin, ...","(pesananku, udah bayar tapi kenapa dibatalin, ...",payment; produk;
1,"(call center, hanya berbelit belit, negative, ...","(call center, hanya berbelit belit, negative, ...",delivery; produk;
2,"(alfa, kalo jumat-minggu ada promo, positive, ...","(alfa, kalo jumat-minggu ada promo, positive, ...",price; produk;
3,"(tokopedia, gangguan, negative, website&apps);","(tokopedia care, sedang gangguan, neutral, web...",website&apps; produk;
4,"(token listrik, lewat tokped error, negative, ...","(token listrik, lewat tokped error, negative, ...",website&apps; payment; produk;


In [19]:
df.head()[['annotated_multilabel', 'model_multilabel', 'keyword_multilabel']]

,annotated_multilabel,model_multilabel,keyword_multilabel
0,{website&apps},{website&apps},"{produk, payment}"
1,"{customerservice, delivery}","{customerservice, delivery}","{produk, delivery}"
2,{price},{price},"{produk, price}"
3,{website&apps},{website&apps},"{website&apps, produk}"
4,{website&apps},{website&apps},"{website&apps, payment, produk}"


In [20]:
y_true = MultiLabelBinarizer(classes=aspect_categories).fit_transform(df['annotated_multilabel'])
y_keyword = MultiLabelBinarizer(classes=aspect_categories).fit_transform(df['keyword_multilabel'])
y_model = MultiLabelBinarizer(classes=aspect_categories).fit_transform(df['model_multilabel'])

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['product'] will be ignored
  warnings.warn(


In [21]:
#for keyword
print(classification_report(y_true,y_keyword, target_names=aspect_categories))

                 precision    recall  f1-score   support

          price       0.67      1.00      0.80        12
         produk       0.00      0.00      0.00         0
        payment       0.25      0.67      0.36         3
   website&apps       1.00      0.67      0.80         6
       delivery       0.81      0.93      0.87        14
customerservice       0.25      0.33      0.29         3

      micro avg       0.36      0.84      0.50        38
      macro avg       0.50      0.60      0.52        38
   weighted avg       0.71      0.84      0.75        38
    samples avg       0.37      0.79      0.50        38



c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
#for model
print(classification_report(y_true, y_model, target_names=aspect_categories))

                 precision    recall  f1-score   support

          price       1.00      0.92      0.96        12
         produk       0.00      0.00      0.00         0
        payment       1.00      1.00      1.00         3
   website&apps       0.86      1.00      0.92         6
       delivery       1.00      1.00      1.00        14
customerservice       1.00      1.00      1.00         3

      micro avg       0.97      0.97      0.97        38
      macro avg       0.81      0.82      0.81        38
   weighted avg       0.98      0.97      0.97        38
    samples avg       0.90      0.90      0.90        38



c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danendra\ana

# Sentiment

In [12]:
df.head(1)

,original_id,content,clean_tweet,final_sentiment,labels,quadruplet,spam,sentiment_label,pred_quadruplet_pt_bart,pred_quadruplet_pt_t5,pred_quadruplet_tf_t5,annotated_multilabel,keyword_multilabel,model_multilabel
0,1.640000e+18,@tokopedia min aku udah bayar tapi kenapa diba...,min aku udah bayar tapi kenapa dibatalin pesan...,negative,payment; produk;,"(pesananku, udah bayar tapi kenapa dibatalin, ...",NaN,sentiment,"(_, kenapa dibatalin pesanan ku, neutral, pay...","(pesananku, udah bayar tapi kenapa dibatalin, ...","(pesanan, dibatalin pesanan, negative, delivery);",{website&apps},"{produk, payment}",{website&apps}


In [16]:
annot_sents = []
keyword_sents = []
model_sents = []

ori_quadruplet_col = 'quadruplet'
pred_quadruplet_col = 'pred_quadruplet_pt_t5'
keyword_sentiment_col = 'final_sentiment'
for i in range(len(df)):
    row = df.iloc[i]
    #for annotated aspect categories
    annotated_quadruplets = extract_quadruplet(row[ori_quadruplet_col])
    annot_sent = []
    #for model aspect categories
    model_quadruplets = extract_quadruplet(row[pred_quadruplet_col])
    model_sent = []
    #for annotated quadruplet
    for quadruplet in annotated_quadruplets:
        #for annotated
        aspect_term, opinion_term, sentiment, aspect_category = post_process(quadruplet)
        annot_sent.append(sentiment)
    #for model quadruplet
    for quadruplet in model_quadruplets:
        #for annotated
        aspect_term, opinion_term, sentiment, aspect_category = post_process(quadruplet)
        model_sent.append(sentiment)
    annot_sent = set(annot_sent)
    model_sent = set(model_sent)
    if len(annot_sent)>1 or len(model_sent)>1:
        continue
    else:
        annot_sents.append(list(annot_sent)[0])
        model_sents.append(list(model_sent)[0])
        keyword_sents.append(row[keyword_sentiment_col])

In [19]:
len(keyword_sents), len(model_sents), len(annot_sents), df.shape[0]

(36, 36, 36, 39)

In [20]:
print(classification_report(y_true=annot_sents, y_pred=keyword_sents))

              precision    recall  f1-score   support

    negative       0.63      0.89      0.74        19
     neutral       0.00      0.00      0.00         6
    positive       0.00      0.00      0.00        11

    accuracy                           0.47        36
   macro avg       0.21      0.30      0.25        36
weighted avg       0.33      0.47      0.39        36



c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
print(classification_report(y_true=annot_sents, y_pred=model_sents))

              precision    recall  f1-score   support

    negative       1.00      0.95      0.97        19
     neutral       0.86      1.00      0.92         6
    positive       1.00      1.00      1.00        11

    accuracy                           0.97        36
   macro avg       0.95      0.98      0.97        36
weighted avg       0.98      0.97      0.97        36

